I use EXP-4 100 rows for testing mlae 

In [1]:
import pandas as pd

balanced_df = pd.read_csv('balanced_data_sample.csv')
#balanced_df.head(10)


Daniel, I found something interesing here

From your code: [run_position_angle.py](https://github.com/haehn/perception/blob/master/EXP/run_position_angle.py),  

I found you are using this calculation:  

```python
MLAE = np.log2(sklearn.metrics.mean_absolute_error(y_pred * 100, y_test * 100) + 0.125)
```

And I think this is the correct calculation for **Cleveland and McGrill**.  

However, at the beginning, you can check this link:  
[LLMP/evaluator.py](https://github.com/kenichi-maeda/LLMP/blob/main/LLMP/evaluator.py)  

I originally used this MLAE calculation for our project:  

```python
mlae = np.log2(mean_absolute_error(gt_array, answers_array) + 0.125)
```

My Thought:  
I think I **did not** use Cleveland and McGrill's MLAE calculation for our project.




In [3]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics

# MLAE Calculation 1: Cleveland-McGill (Log Transformed MAE)
def Cleveland_McGill(y_pred, y_test):
    return np.log2(metrics.mean_absolute_error(y_pred * 100, y_test * 100) + 0.125)

# MLAE Calculation 2: Per-Instance Log Transformed MAE
def MLAE_per_instance(y_pred, y_test):
    mlae_values = [
        np.log2(metrics.mean_absolute_error([gt], [pred]) + 0.125) 
        for gt, pred in zip(y_test, y_pred)
    ]
    return np.mean(mlae_values)  # Average across all instances

# Load dataset
df = balanced_df.copy()

# Convert columns to numeric
df['ground_truth'] = pd.to_numeric(df['ground_truth'], errors='coerce')
df['cleaned_answers'] = pd.to_numeric(df['cleaned_answers'], errors='coerce')

# Drop missing values
df = df.dropna(subset=['ground_truth', 'cleaned_answers'])

# Compute MLAE for each model
mlae_results = {}
for model, group in df.groupby('model_name'):
    y_test = group['ground_truth'].values
    y_pred = group['cleaned_answers'].values

    mlae_Cleveland_McGill = Cleveland_McGill(y_pred, y_test)
    mlae_per_instance = MLAE_per_instance(y_pred, y_test)

    mlae_results[model] = (mlae_Cleveland_McGill, mlae_per_instance)

    # Display Example Calculation for First 5 Rows
    print(f"\nExample Calculation for Model: {model}")

    # Take the first 5 rows for demonstration
    example_y_test = y_test[:5]
    example_y_pred = y_pred[:5]

    print(f"Ground Truth: {example_y_test}")
    print(f"Predictions: {example_y_pred}")

    # Step-by-step for Cleveland-McGill MLAE
    example_mae = metrics.mean_absolute_error(example_y_pred * 100, example_y_test * 100)
    log_mae = np.log2(example_mae + 0.125)

    print(f"Step-by-Step Cleveland-McGill MLAE Calculation:")
    print(f"Mean Absolute Error (MAE) * 100: {example_mae * 100:.4f}")
    print(f"MLAE = log2({example_mae * 100:.4f} + 0.125) = {log_mae:.4f}")

    # Step-by-step for Per-Instance MLAE
    per_instance_values = [
        np.log2(metrics.mean_absolute_error([gt], [pred]) + 0.125) 
        for gt, pred in zip(example_y_test, example_y_pred)
    ]

    print(f"\nStep-by-Step Per-Instance MLAE Calculation:")
    for i, (gt, pred, per_mlae) in enumerate(zip(example_y_test, example_y_pred, per_instance_values)):
        print(f"Instance {i+1}: GT = {gt}, Pred = {pred}, MLAE = log2(|{gt} - {pred}| + 0.125) = {per_mlae:.4f}")

    print(f"Average Per-Instance MLAE: {np.mean(per_instance_values):.4f}")

# Display final results for all models
print("\nMLAE Comparison (Cleveland-McGill vs. Per-Instance MLAE):")
for model, (mlae_Cleveland_McGill, mlae_per_instance) in mlae_results.items():
    print(f"{model}: Cleveland-McGill MLAE = {mlae_Cleveland_McGill:.4f}, Per-Instance MLAE = {mlae_per_instance:.4f}")



Example Calculation for Model: CustomLLaMA
Ground Truth: [6 8 9 8 8]
Predictions: [ 9. 10.  3.  3. 10.]
Step-by-Step Cleveland-McGill MLAE Calculation:
Mean Absolute Error (MAE) * 100: 36000.0000
MLAE = log2(36000.0000 + 0.125) = 8.4924

Step-by-Step Per-Instance MLAE Calculation:
Instance 1: GT = 6, Pred = 9.0, MLAE = log2(|6 - 9.0| + 0.125) = 1.6439
Instance 2: GT = 8, Pred = 10.0, MLAE = log2(|8 - 10.0| + 0.125) = 1.0875
Instance 3: GT = 9, Pred = 3.0, MLAE = log2(|9 - 3.0| + 0.125) = 2.6147
Instance 4: GT = 8, Pred = 3.0, MLAE = log2(|8 - 3.0| + 0.125) = 2.3576
Instance 5: GT = 8, Pred = 10.0, MLAE = log2(|8 - 10.0| + 0.125) = 1.0875
Average Per-Instance MLAE: 1.7582

Example Calculation for Model: Gemini1_5Flash
Ground Truth: [6 8 9 3 8]
Predictions: [7. 7. 7. 7. 7.]
Step-by-Step Cleveland-McGill MLAE Calculation:
Mean Absolute Error (MAE) * 100: 18000.0000
MLAE = log2(18000.0000 + 0.125) = 7.4929

Step-by-Step Per-Instance MLAE Calculation:
Instance 1: GT = 6, Pred = 7.0, MLAE =

Continue to understand why? https://chatgpt.com/c/67a0bc52-4948-800b-b745-465bb11f566f 